# Clustering

## Clustering as unsupervised learning

Clustering is an application of machine learning that pervades biology as well as numerous other domains. The question clustering answers is simple: given a large number of data points, can we group these points so that they share common properties? Moreover, exactly what properties do they share? For example, in this portion of the workshop, we will be clustering cells based on gene expression data. Rather than examining each cell individually, we want to understand the primary groups into which the cells can be separated.

Clustering is an example of *unsupervised learning*, in which we don't have the "correct" answer for training data with which we can establish our model's accuracy. While we sometimes have a ground truth against which we can compare our results, more often we don't know what the proper answer is. Indeed, many possible clusterings are possible for a given dataset, depending on what algorithm and parameters you use, and which is best depends on the question you hope to answer. For instance, in clustering cells by gene expression data, sometimes we will want only three clusters that broadly summarize trends in the underlying data; in other cases, we might want 100 clusters, in which each cluster is smaller and more homogeneous in its properties. Which we prefer will depend on what biological question we want to address.

## The problem: clustering gene expression data

[Trapnell (2014)](http://www.nature.com/nbt/journal/v32/n4/fig_tab/nbt.2859_F1.html) performed single-cell RNA-seq on differentiating skeletal myoblasts at 0, 24, 48, and 72 hours. The original data are available via [GEO database accession number GSE52529](ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE52nnn/GSE52529/suppl/GSE52529_fpkm matrix.txt.gz). We have at hand a cleaned-up version from [Sincell](https://www.bioconductor.org/packages/3.3/bioc/vignettes/sincell/inst/doc/sincell-vignette.pdf), in which only the expression levels for 575 differentially-expressed genes have been retained across 271 cells drawn from the various timepoints.

Our goal is to cluster the 271 cells based on gene expression. *Clustering* is simply a means of grouping related items in your data together, allowing you to gain insight into overarching trends. In this case, we expect cells at each timepoint to have relatively similar expression levels across different genes, meaning that we hope cells from the same timepoints will cluster together. In this regard, knowing what timepoint each cell comes from provides us with a "ground truth" means of evaluating our clustering&mdash;if we see cells from the same timepoint present in the same cluster, it means our clustering is working as intended. This will give us confidence in applying our clustering methods to other single-cell datasets, where we hope to discover underlying structure based on factors other than time since cell division.

## Our first clustering algorithm: k-means

K-means is an extremely simple clustering algorithm that is nonetheless quite effective.

1. Decide how many clusters $k$ you wish to use.
2. Place $k$ cluster centres ("centroids") at random locations in the space in which your data lives.
3. Assign each data point to the cluster corresponding to its closest centroid.
4. Repeat:
    1. For each cluster, move its centroid to the centre (i.e., mean) of all points belonging to the cluster.
    2. Assign each data point to its closest centroid.
    3. If no points changed their cluster assignments, exit. Otherwise, return to step 4.

The following example illustrates the algorithm. Cluster centroids are circles, while data points are squares. Here, we have arbitrarily chosen to create $k=3$ clusters.

| Step | Illustration |
| ---- | ------------ |
| Step 1: Place $k=3$ centroids randomly. Here, we have arbitrariliy chosen $k=3$&mdash;we could have easily used a different value of $k$. | ![K-means step 1](images/kmeans_example_step1.svg) |
| Step 2: Assign each data point to its closest centroid. | ![K-means step 2](images/kmeans_example_step2.svg) |
| Step 3: Move each centroid to the centre of all data points belonging to its cluster. | ![K-means step 3](images/kmeans_example_step3.svg) |
| Step 4:Reassign each data point to the cluster with the closest centroid. | ![K-means step 4](images/kmeans_example_step4.svg)

(Images taken [from Wikipedia](https://en.wikipedia.org/wiki/K-means_clustering#/media/File:K_Means_Example_Step_1.svg).)

We repeat these steps until convergence&mdash;that is, when no points change cluster assignments. Note that the precise solution you obtain will depend on the random locations you initially placed your centroids. Rerunning the algorithm with different starting points for the centroids may change the cluster assignments of some points.

## Clustering simulated data via k-means

Now that we understand how k-means works, let's try running it on a test dataset. We will create three different two-dimensional Gaussian distributions with different means, then sample points from each. As we know what Gaussian we drew each point from, this gives us the ground-truth class for each point against which we can compare our clustering results.

In [24]:
from cluster import plot_simulated
import numpy as np
import sklearn.cluster

np.random.seed(1)

def generate_simulated_points():                                                                                             
  num_classes = 3                                                                                                            
  points_per_class = 40                                                                                                      
  points = np.zeros((num_classes * points_per_class, 2))                                                                     
  labels = np.zeros(num_classes * points_per_class)                                                                          
  means = [(-3, -2), (-2, -4), (3, 2)]                                                                                       
  cov = [[0.8, 0.2], [3, 0.5]]                                                                                                           
                                                                                                                             
  for idx in range(num_classes):                                                                                             
    rows = range(idx * points_per_class, (idx + 1) * points_per_class)                                                       
    points[rows,:] = np.random.multivariate_normal(means[idx], cov, points_per_class)                                        
    labels[rows] = idx                                                                                                       
                                                                                                                             
  idxs = np.random.permutation(np.arange(num_classes * points_per_class))                                                    
  return (points[idxs,:], labels[idxs])  

sim_points, sim_labels = generate_simulated_points()
sim_preds_kmeans = sklearn.cluster.KMeans(n_clusters=3).fit_predict(sim_points)
plot_simulated(sim_points, sim_preds_kmeans, sim_labels, 'k-means')

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:17: RuntimeWarning:

covariance is not positive-semidefinite.



In this plot, the ground-truth class is represented by the *shape* of point (triangle, plus sign, or star), while the cluster to which our algorithm assigned it is represented by the *colour* of the point (blue, red, or orange). Observe that the clustering assignments are largely consistent with the ground truth labels, meaning our clustering is accurate. This only breaks down when two clusters overlap, as with the red and blue clusters&mdash;there, we have some blue plus signs and some red triangles, which are incorrectly clustered. Note that there's no way for our clustering algorithm to determine the correct assignments in these cases&mdash;without information about each point beyond just its position in space, we cannot expect our clustering algorithm to recover the correct answer.

## Our second clustering algorithm: fitting a mixture of Gaussians using expectation maximization

As an alternative to k-means, we can fit a *mixture of Gaussians* using *expectation maximization*. (You'll also see references to *GMMs* below, which are the same thing&mdash;*GMM* stands for *Gaussian mixture model*.) Let's first discuss what a mixture of Gaussians is. Imagine you have a bunch of points that you know were sampled from different Gaussian probability distributions.

![MOG truth](images/mog_truth.png)

Here, you see points drawn from three different Gaussians. Each Gaussian has parameters describing its mean and variance, which determine the position and shape of the distribution. As there is more probability mass under the "centre" of the Guassians than under the "tails", we see more points sampled under the centre of each distribution. But, of course, if you were to encounter this data in the real world, you wouldn't know the parameters of the Gaussians, so you would have no idea what the distributions look like.

![MOG truth](images/mog_no_pdfs.png)

In fact, you don't even know which Gaussian each point was sampled from.

![MOG truth](images/mog_no_labels.png)

However, to reduce the complexity of the problem, let's assume for the moment that you do know that the data was drawn from Gaussian distributions, however, and that there were originally three Gaussians. Given this knowledge, how do you determine the parameters for each Gaussian, and consequently what distribution each point belongs to? The answer is expectation maximization.

Expectation maximization is an iterative algorithm that lets you establish both the parameters of the distributions from which your data was generated, as well as the most likely assignments of points to distributions. As such, in our example, we can think of each Gaussian corresponding to a cluster. To determine the cluster parameters and assignments, we use the following algorithm:

1. E (expectation) step: Each cluster has associated with it parameters describing its properties. For our Gaussian example, this corresponds to a mean and variance for each Gaussian describing the position and shape of the distribution. For each data point, determine the *likelihood* that it was generated by each cluster.

2. M (maximization) step: For each data point and each cluster, update the parameters of the cluster based on the points assigned to it.

3. If the cluster assignments and parameters hardly changed (i.e., converged) in the E and M steps, exit. Otherwise return to step 1.

![MOG all steps](images/mog_all_steps.png)

Here we observe this process in action.

* In step 1, we initialize three Gaussians with random means and variances.
* In step 2, we have performed our first E and M steps. Each point is associated with its most likely Gaussian (E step); each Gaussian is adjusted to better fits its associated points (M step).
* In step 3, we perform another run of the E and M steps.
* In step 4, the algorithm has converged and so we terminate. The answer we arrive at precisely matches the ground truth.

What is not captured by the image is that, unlike k-means, expectation maximization generates *soft* rather than *hard* assignments of points to clusters. K-means' assignments are "hard" because, at each step, every point is assigned with certainty to one and only one cluster. Expectation maximization, conversely, generates "soft" assignments, as every point is assigned with some probability to every cluster. This has two consequences. Firstly, the higher the probability of a point's assignment to a given cluster, the more "weight" that point will have in determining that cluster's parameters. For example, in step 2 of our EM example, the points directly under the centre of the red Gaussian will have a large influence in determining the updated mean of the distribution for step 3. But even the green points on the far right of the axis will have a small influence on the new mean of the red Gaussian, in accordance with their infinitesimal probability of having been generated by the red Gaussian. Once the algorithm converges, we can get hard assignments of points to clusters by simply taking the most probable cluster for each point.

Relative to k-means, expectation maximization is a more general algorithm that can be used for purposes other than clustering. In fact, EM can be applied to a wide class of problems in which we confront *latent* (hidden) variables that affect our inference. In this example, the latent variables (or hidden knowledge) are which distribution generated each point. But you can also apply EM to diverse other problems. Imagine you were flipping two coins, each with some unknown bias towards or tails. Given the results of 100 coin flips (heads, tails, heads, heads, ...), in which one coin or the other was selected randomly at each step, you want to know which coin was flipped at each step, and what bias each coin has. This problem appears quite difficult, as you lack knowledge not only of the bias parameter associated with each coin, but also the identity of the coin used for each flip. Expectation maximization gives you a principled way of determining the most likely solution to this problem.

## Clustering simulated data via EM on a mixture of Gaussians

Now that we understand we can cluster data using expectation maximization on a mixture of Gaussians, let's try running it on a test dataset.

In [2]:
sim_preds_gmm = sklearn.mixture.GMM(n_components=3).fit_predict(sim_points)
plot_simulated(sim_points, sim_preds_gmm, sim_labels, 'EM on mixture of Gaussians')

Let's compare to our previous results on k-means.

In [3]:
plot_simulated(sim_points, sim_preds_kmeans, sim_labels, 'k-means')

We can see that the two algorithms produce largely comparable results, but differ on a few points. For example, examine the two "plus sign" points around $(x,y)=(-\frac{1}{3},-2)$. We can see that, under EM, both are assigned to the red cluster, but under k-means, one goes to red and one goes to blue.

## Evaluating clustering success: extrinsic metric via V-measure

In determining whether our clustering was successful, we can draw upon both extrinsic and intrinsic measures. Extrinsic measures are similar to supervised learning&mdash;given knowledge of the ground truth concerning what classes each of your data points belongs to, you can determine how well a clustering recapitulates that structure. Consequently, on data for which you do not have the ground truth, so long as the properties of that data echo that of the data for which you know the classes, you assume that your clustering method works equally well. Intrinsic measures, by comparison, work even when you don't have ground-truth class assignments to compare against. Instead, intrinsic measures draw solely on properties of the clustering.

We will first discuss V-measure, an extrinsic means of evaluating clusterings. While numerous other measures exist, V-measure has a wonderfully intuitive explanation, as it measures the *homogeneity* and *completeness* of a clustering. Homogeneity is maximized when all points in a cluster are drawn from the same ground-truth class, while completeness is maximized when all points in a class are assigned to only one cluster. Thus, a good clustering will optimize both of these metrics.

To gain an intution for homogeneity and completeness, let's consider several scenarios. In a perfect clustering&mdash;that is, your clustering exactly recapitulates the true class structure&mdash;both homogeneity and completeness will reach their maximum values. However, in imperfect clusterings, homogeneity and completeness are often at odds with each other, as efforts to increase one will decrease the other. Consider a degenerate case in which every point is assigned to its own cluster, such that, given 271 cells, we have 271 clusters. Here, homogeneity is maximized, as each cluster indeed contains only one class of point; completeness, however, is extremely poor, as in our RNA-seq example, every cell from a given timepoint would be in a separate cluster. There's also a reciprocal degenerate case that maximizes completeness&mdash;imagine we placed every point in a single cluster. Then every cell from a given timepoint would indeed be in the same cluster as the other cells from that timepoint, maximizing completeness; but homogeneity would be extremely poor, as that cluster would contain a mixture of cells from *every* timepoint.

Both homogeneity and completeness take values ranging from zero to one, with one indicating the best possible performance on the measure. To evaluate homogeneity and completeness, we formulate these concepts in terms of *entropies*. Entropy represents the amount of uncertainty exhibited by a given random variable. Suppose you flip a coin that comes up heads 99% of the time. This coin has low entropy, as you have minimal uncertainty about what the result will be on each flip. Conversely, if you flip a perfectly fair coin, you have no idea whether it will come up heads or tails, and so this coin has high entropy. To work with V-measure, we need two notations for entropy:

* $H(X)$ is the entropy (uncertainty) associated with the random variable $X$. In the example of our completely fair coin, $H(X)$ is a value indicating the uncertainty as the outcome of each coin toss.
* $H(X|Y)$ is the conditional entropy of $X$, assuming you already know the value of $Y$. Returning to our coin example, imagine $Y$ is a binary variable telling you whether you chose the fair or unfair coin. Then $H(X|Y)$ is the average uncertainty in the results of my coin tosses, assuming I know what the probability is that I picked one coin or the other.

Given $N$ points in your dataset, let $K=[k_1,k_2,...,k_i,...,k_N]$, such that the $i$th element indicates the identity of the cluster to which point $i$ belongs. Likewise, let $C=[c_1,c_2,...,c_i,...,c_N]$ indicate the ground-truth class of point $c_i$. Using these two random variables, we can define homogeneity and completeness.

We will define the homogeneity $h$ thusly:

$h=1-\frac{H(C|K)}{H(C)}$

Let's first think about the quantity $\frac{H(C|K)}{H(C)}$. The value $H(C)$ is a measure of uncertainty in the class distribution&mdash;the more classes you have, and more equal they are in the number of members belonging to each class, the higher the uncertainty as to which class any given point comes from. Comparing this to $H(C|K)$, then, tells you how much your uncertainty about class membership decreases when I tell you what clusters each point belongs to. For example, $H(C|K)=0$ if I ask you "What class do the points in the third cluster belong to?", and you can answer with no uncertainty because all the points in that cluster come from only a single class. In this scenario, $h=1-\frac{H(C|K)}{H(C)}=1-0=1$, meaning you have perfect homogeneity. But note that $h=1$, meaning that there is only one class for the points in a given a cluster, can occur in both of the scenarios we previously discussed&mdash;you get perfect homogeneity if your clustering perfectly matches your class structure, or if you simply place each point in its own class. Conversely, $h=0$ when $H(C|K)=H(C)$, meaning that knowing what points are in a cluster gives you no information whatsoever about the classes from which those points originate, as there is no concordance between the clustering and underlying class structure.

Now, let's think about the completeness $c$, defined thusly:

$c=1-\frac{H(K|C)}{H(K)}$

Similar araguments apply as for homogeneity. $H(K)$ is the uncertainty in the class distribution, with datasets possessing more classes and more equitable distribution of points between classes having higher uncertainties. Comparing this to $H(K|C)$, then, tells you how much uncertainty about what cluster points belong to decreases when you know what class those points come from. If I ask "What cluster are the cells from the $T=0$ timepoint in?" and you can answer with no uncertainty because the cells are all in the same cluster, then $H(K|C)=0$, and so $c=1$.. But again, this occurs either under a clustering that perfectly echoes the class structure, or in the degenerate case when all points are assigned to a single cluster regardless of their original class. But if knowing the classes of some of your datapoints tells you nothing about what clusters they were assigned to, then there's no concordance between clusters and classes, then $H(K|C)=H(K)$, and so $c=0$.

Finally, to calculate the V-measure score $v$, we take the harmonic mean of the completeness $c$ and homogeneity $h$:

$v=\frac{2}{\frac{1}{h}+\frac{1}{c}}=\frac{2hc}{h+c}$

V-measure thus represents the average of the completeness and homogeneity, reflecting both scores in a single value. We prefer the harmonic mean to the arithmetic mean because the harmonic mean punishes more harshly poor values for either metric. Suppose we encountered our degenerate case where all points are assigned to a single class, such that $c=1$ but $h=0$. Then, the arithmetic mean is $0.5$; the geometric mean, however, is zero. (Well, strictly speaking, it is undefined, but V-measure establishes $v=0$ when $h=0$ or $c=0$ as a special case.)

More rigorously, we use the harmonic mean because we can instead define $h=\frac{MI(K,C)}{H(C)}$ and $c=\frac{MI(K,C)}{H(K)}$, where $MI(K,C)$ is the mutual information shared between $K$ and $C$. As both $h$ and $c$ share their numerators in this case, the harmonic mean is the most sound method of taking their average. This is, however, beyond the scope of this workshop.

## Evaluating clustering success: Intrinsic metric via silhouette score

Evaluating clustering quality via an extrinsic metric such as V-measure requires you have data accompanied by a ground-truth listing of the proper classes to which each point belongs. In the absence of such information, you must instead rely on intrinsic measures that determine quality without reference to external information.

Silhouette is one intrinsic measure by which you can evaluate clustering. Recall that we established the distance between cells using Euclidean distance. For each datum $i$ you have clustered, let $a(i)$ be the average dissimilarity between $i$ and every other member of its cluster. In our case, our dissimilarity metric is the Euclidean distance between cells. Now let $b(i)$ be the average dissimilarity between $i$ and the points in the closest clustering neighbouring the one to which $i$ belongs. Given these values, we can define the silhouette $s(i)$:

$s(i) = \frac{b(i) - a(i)}{\max\{a(i),b(i)\}}$

Equivalently, we can write the following:

$s(i) = \begin{cases}
  1-a(i)/b(i), & \mbox{if } a(i) < b(i) \\
  0,  & \mbox{if } a(i) = b(i) \\
  b(i)/a(i)-1, & \mbox{if } a(i) > b(i) \\
\end{cases}$

From this, we can see $-1 \le s(i) \le 1$. Consequently, we can conclude two things about $i$:

* If $s(i)$ is close to $1$, then $a(i) \ll b(i)$. This means $i$ is clustered well, as it's closely related to other members of its cluster, but is distant from the next-best-fitting cluster.
* If $s(i)$ is close to $-1$, then $a(i) \gg b(i)$, implying $i$ is clustered poorly&mdash;$i$ would fit better in its neighboring cluster than in the one in which it is present.

Of course, we have thus far defined the silhouette only for a single point $i$. To gain an understanding of how well-clustered the dataset is as a whole, we can calculate the mean silhouette score for every datum. Alternatively, to understand the quality of each cluster, we can calculate the mean silhouette score for each cluster individually.

## Evaluating clustering success for our simulated data

Let's first define a `print_metrics` helper function to calculate and print our various measures of cluster quality.

In [4]:
import sklearn.metrics

def print_metrics(data, classes, clusters):
  homogeneity, completeness, v_measure = sklearn.metrics.homogeneity_completeness_v_measure(classes, clusters)
  print('Homogeneity, completeness, v_measure:', (homogeneity, completeness, v_measure))
  print('Mean silhouette score:', sklearn.metrics.silhouette_score(data, clusters))

Now let's evaluate our k-means results.

In [5]:
print_metrics(sim_points, sim_labels, sim_preds_kmeans)

Homogeneity, completeness, v_measure: (0.75895660773022333, 0.76025535361635044, 0.75960542553650556)
Mean silhouette score: 0.552423602656


We'll do the same for our GMM results.

In [6]:
print_metrics(sim_points, sim_labels, sim_preds_gmm)

Homogeneity, completeness, v_measure: (0.69677160493746149, 0.69730075095729649, 0.69703607752378571)
Mean silhouette score: 0.542725996296


Both k-means and the GMM do reasonably well by these metrics. In both models, homogeneity and completeness are nearly the same, meaning neither is favoured by k-means or the GMM.

In comparing the k-means results to the GMM ones, we see that k-means does slightly better on all four metrics. This may be because of the simplicity of the model that generated our data. K-means implicitly assumes that data were generated from isotropic (spherical) Gaussians, meaning their covariance matrices are simply the identity matrix, potentially multiplied by a scalar. The Gaussian mixture model, conversely, is a more complex model that learns all the parameters of the underlying Gaussians, including the covariance matrices. Recall that the covariance matrices used to generate our data were simply the identity matrix&mdash;the underlying Gaussians differed only in their means. As two of our clusters are overlapping in the data we generate, the GMM may try to account for this with more complicated covariance matrices; the simplicity of k-means, by contrast, better matches the manner in which we generated the data, potentially leading to a more accurate model.

## Exercise: Generate more complex simulated data and evaluate performance

Now it's your turn. Adjust how we generate the simulated data so that the underlying Gaussians use different, full-rank covariance matrices, meaning that all values on the diagonal are different (making traces of the Gaussian elliptical rather than strictly circular), and so off-diagonal values are non-zero and different from one another (rotating the Gaussian so it is no longer axis-aligned). See if this leads to the GMM exhibiting better performance than k-means.

In [7]:
# Your brilliant, world-changing, paradigm-defining code goes here.

## Choosing the best number of clusters

When clustering, choosing the best number of clusters is difficult. Often there is no correct answer&mdash;the "best" number of clusters will depend on the purpose to which you will put the results. We can, however, examine intrinsic properties of the results to inform our decision.

First, we will define a function for calculating the sum of squared errors. This value is simply the sum of each point's squared Euclidean distance from its cluster centre. By squaring the distance, we punish points lying far from their centres more harshly.

In [8]:
def calc_sum_sqerror(classifier, points, preds):
  sum_sqerror = 0
  for pred, point in zip(preds, points):
    cluster_centre = classifier.cluster_centers_[pred]
    dist = np.sum((point - cluster_centre)**2)
    sum_sqerror += dist
  return sum_sqerror

Now, we can try running k-means with the number of clusters $K=[1,2,...,10]$, then evaluate various metrics to see how each $K$ performs. First we will run k-means and record several metrics for each $K$.

In [9]:
from cluster import plot_line_chart

# cluster_counts must be JSON-serializable for Plotly, so convert to list.
cluster_counts = list(range(1, 11))
sqerrors = []
silhouettes = []
vmeasures = []

for K in cluster_counts:
  kmeans = sklearn.cluster.KMeans(n_clusters=K)
  preds = kmeans.fit_predict(sim_points)
  sqerrors.append(calc_sum_sqerror(kmeans, sim_points, preds))
  vmeasures.append(sklearn.metrics.v_measure_score(sim_labels, preds))
  if K > 1:
    # Need at least two clusters to calculate silhouette.
    silhouettes.append(sklearn.metrics.silhouette_score(sim_points, preds))

Next, we will plot the sum of squared errors as a function of $K$.

In [10]:
plot_line_chart(
  cluster_counts,
  sqerrors,
  'Squared error as function of number of clusters',
  'Number of clusters',
  'Squared error'
)

As expected, the sum of squared errors decreases monotonically&mdash;adding an additional cluster will *always* decrease the total error. Our goal is to find the "elbow" on the curve corresponding to the "best" value of $K$. For this $K$ value, we should see susbstantially lower error relative to $K-1$, but at $K+1$, we should observe that error decreases far less dramatically relative to $K$. On this curve, though we know the "correct" number of clusters is three, the clearest elbow occurs at $K=2$. This is likely because two of the clusters partially overlap, meaning that this metric prefers only two distinct clusters. This demonstrates how difficult it can be to determine the "correct" number of clusters, given that on most datasets we don't know what the "correct" answer is.

Next, we'll see how mean silhouette scores change with the number of clusters. Note that silhouette scores are only defined for $K>1$, so we must exclude $K=1$ when plotting these values.

In [11]:
plot_line_chart(
  cluster_counts[1:],
  silhouettes,
  'Mean silhouette score as function of number of clusters',
  'Number of clusters',
  'Silhouette score'
)

Unlike sum-of-squared-errors, silhouette scores do not decrease monotonically. Here, we again prefer the "wrong" $K=2$, likely because of the overlapping clusters.

Finally, we will evaluate our simulated results using V-measure. Both sum-of-squared errors and silhouette scores are intrinsic measures, meaning we can use them even without a ground-truth listing of the correct class for each point. Using V-measure, however, is possible here only because we have such labels.

In [12]:
plot_line_chart(
  cluster_counts,
  vmeasures,
  'V-measure score as function of number of clusters',
  'Number of clusters',
  'V-measure score'
)

Interestingly, we see here a slight preference for $K=3$ rather than $K=2$, with the respective V-measure scores being 0.76 and 0.73. Again, however, we can recover the "correct" $K$ only because we have access to ground-truth labels that show the one apparent big cluster should in fact be separated into two separate clusters. Without these labels, our model prefers only two clusters, as demonstrated by the two intrinsic measures.

## Exercise: Make distinct clusters and see if you recover the "correct" value of $K$

As two of our clusters overlap, our various intrinsic means of choosing $K$ have preferred $K=2$ clusters, rather than the correct value of $K=3$. Try changing the means of the Gaussians generating the clusters to make the clusters distinct, then see if the various measures recover the "correct" $K$.

Additionally, you can evaluate the Gaussian mixture model as well as k-means for different values of $K$ on the simulated data. Use sum-of-squared-error, silhouette scores, and V-measure to choose the best value, and to see if it matches the value used to generate the data.

In [13]:
# Your most excellent code goes here. I mean, seriously, don't disappoint us.
# I'm expecting to see the Python equivalent of Hemingway here.

## Preparing to cluster our gene expression data

We have a 271x575 matrix containing gene expression levels. Each row corresponds to a cell, while each column corresponds to the expression of a gene in $\log\text{FPKM}$ units. We will perform dimensionality reduction via PCA on our data before clustering to avoid the *curse of dimensionality*, which we will discuss next.

In [14]:
from cluster import get_timepoints, plot_expression
import sklearn.decomposition

def load_exprmat(exprmat_fn):
  rows = []
  rownames = []

  with open(exprmat_fn) as exprmat:
    colnames = next(exprmat).strip().split(',')
    for row in exprmat:
      fields = row.split(',')
      rownames.append(fields[0])
      rows.append([float(f) for f in fields[1:]])

  data = np.array(rows)
  return (data, colnames, rownames)

def reduce_dimensionality(exprmat, n_components):
  pca = sklearn.decomposition.PCA(n_components=n_components)
  projected = pca.fit(exprmat).transform(exprmat)
  print('Explained variance ratio: %s' % pca.explained_variance_ratio_)
  return projected

In [15]:
exprmat, genes, samples = load_exprmat('../data/expression_matrix.csv')
print('Matrix size:', exprmat.shape)
timepoints = get_timepoints(samples)
projected = reduce_dimensionality(exprmat, 2)

Matrix size: (271, 575)
Explained variance ratio: [ 0.22354609  0.09390967]


There are three important variables you will work with from the above.

  * `projected` is the 271x575 gene expression matrix `exprmat` projected via PCA so it becomes 271x2. We project it from 575 dimensions corresponding to different genes, to only two dimensions corresponding to two principal components composed of the genes responsible for the most variance in the underlying data. See the next section for why we must do PCA on the original data.
  * `samples` is a 271-element list giving the name of each cell. Each name contains the cell's timepoint, which we extract in `timepoints` to give us ground-truth classes.
  * `timepoints` is a 271-element vector listing the ground-truth class for each cell. Each timepoint is an integer between 0 and 3, corresponding to 0 hours, 24 hours, 48 hours, and 72 hours, respectively.
  
Additionally, observe that our first principal component retains 22% of the original data's variance, while the second principal component adds an additional 9%.

## Avoiding the curse of dimensionality

We will first cluster the cells using the k-means algorithm. K-means, alongside many other clustering algorithms, requires that you have a distance metric such that you can determine the distance between any two cells. Given our dataset of expression levels for each of 575 genes amongst all cells, we can think of each cell as a point in 575-dimensional space. We can thus calculate the distance $d(c_1, c_2)$ between two cells $c_1$ and $c_2$ using Euclidean distance, just as we would for points lying on the Cartesian plane in two dimensions:

$d(c_{1},c_{2})=\sqrt{\sum_{i=1}^{575}\left(c_{1}^{(i)}-c_{2}^{(i)}\right)^{2}}$

But working in 575-dimensional space requires thinking about difficulties we don't encounter in the three dimensions of everyday life. Specifically, when working in so many dimensions, we must confront the *curse of dimensionality*. A [wonderfully intuitive explanation of the curse of dimensionality comes from Kevin Lacker](https://www.quora.com/What-is-the-curse-of-dimensionality/answer/Kevin-Lacker):

> Let's say you have a straight line 100 yards long and you dropped a penny somewhere on it. It wouldn't be too hard to find. You walk along the line and it takes two minutes.
>
> Now let's say you have a square 100 yards on each side and you dropped a penny somewhere on it. It would be pretty hard, like searching across two football fields stuck together. It could take days.
>
> Now a cube 100 yards across. That's like searching a 30-story building the size of a football stadium. Ugh.
>
>The difficulty of searching through the space gets a *lot* harder as you have more dimensions. You might not realize this intuitively when it's just stated in mathematical formulas, since they all have the same "width". That's the curse of dimensionality. It gets to have a name because it is unintuitive, useful, and yet simple.

So, when you're working in 575-dimensional space, you have an incredible volume through which you must search, such that each of your 271 cells is likely to be extremely distant from every other cell. Clustering a cell based on which other cells are "near" and which are "distant" becomes meaningless in this scenario.

What is the solution? Simple! Apply dimensionality reduction before clustering to move your points into a much lower-dimensional space, restoring intuitive notions of "near" and "far". In this case, we will use PCA on our gene expression data before clustering to do exactly that. We will project to only two dimensions to allow us to easily plot the data. You could just as easily, however, use PCA to project the data into ten dimensions, for example. How many dimensions you choose will depend on how much of the original data's variance you want to retain.

## Clustering our gene expression data using k-means

Now we will run k-means on the dimensionality-reduced data. We hope to see our clusters correspond to the timepoint assignment of each cell, such that cells in the same cluster come from the same timepoints. We will show you a basic example, then let you play with the data.

In [16]:
# Remember:
#   * There are four timepoints in the original dataset, so we'll start by trying to fit four clusters.
#   * "projected" is the 575x2 matrix of gene expressions, with reduced dimensionailty via PCA.
#   * "samples" is a 271-element list listing the sample name for each cell.
#   * "timepoints" is a 271-element vector giving the timepoint for each cell.

kmeans_preds = sklearn.cluster.KMeans(n_clusters=4).fit_predict(projected)

Nothing should have been output by the above code. This is good&mdash;it means everything worked!

Now prepare to have your socks blown off. We created a snazzy plotting function for you that lets you compare your results to the ground-truth labels.

In [17]:
plot_expression('k-means', projected, kmeans_preds, timepoints, samples)

The right plot shows your PCA-projected cells, coloured by how they were clustered. The left plot shows the same data, but coloured according to the ground-truth classes. Hover over points in either plot to see the sample names associated with each cell. You can see how difficult it is to cluster these data correctly&mdash;at least when projected to two dimensions via PCA, there's a great deal of intermixing amongst data, making it impossible for any clustering algorithm to recover a fully correct solution.

Why is there so much intermixing between classes on the ground-truth *Timepoints* plot? As the original Trapnell (2014) paper discusses, we don't necessarily see cells at the same timepoint going through coordinated changes in gene expression as they progress together through differentiation. Even though each cell's gene expression was determined via single-cell sequencing, each single cell was drawn from a well containing thousands of cells from the same timepoint. Expression of these cells differed, however, based on where the cell was in the well. If the one sequenced cell was taken from near the centre of the well, it was surrounded by cells on all sides throughout its development, meaning it was subject to a great deal of intercellular communication pushing it to complete differentation quickly. Conversely, if the one sequenced cell was taken from the edge of the well, it received far fewer differentiation signals because it was surrounded by relatively few cells, meaning its progress through differentation was slower. Thus, a "centre" cell may have finished differentiating in 36 hours, for example, while an "edge" cell may have still been early in the differentiation process at the final 72-hour timepoint.

## Exercise: Clustering our gene expression data via Gaussian mixture models

Even though our gene expression were not drawn from a Gaussian distribution, as in the case of our simulated data, a mixture of Gaussians may nevertheless describe the data well. Using what we've shown you, run GMM on the PCA-projected expression data.

In [18]:
# All right, here's your chance to make us proud.

## Exercise: Put everything you've learned into practice

Let's take stock of where we are. You have a cool dataset, a newly stocked machine-learning toolbox full of dapper clustering algorithms, and enthusiastic instructors willing to attend to your every need. This is your chance to do great science!

Here are a few suggestions:

* Evaluate your k-means and GMM results on the gene expression data using homogeneity, completeness, V-measure, and silhouette scores. How do the algorithms compare?

* Even though you know there were originally four timepoints in the data, try clustering with values for $K$ (that is, the number of clusters) other than four. How does this affect the above scoring metrics? How do the V-measure and silhouette score plots look for different values of $K$? Can you find an "elbow" on the sum-of-squared-error plot to determine the "best" number of clusters?

* See how the curse of dimensionality affects your results. Rather than cluster on the PCA-projected 271x2 `projected` matrix, use the original 271x575 `exprmat` matrix. Do our scoring metrics get worse or better? Was the difference less or greater than you expected? Why might this be?

In [19]:
# THIS IS GOING TO BE LEGENDARY!!!